# SemanticChunker: 의미 기반 텍스트 분할 기법

## 개요

**SemanticChunker**는 단순한 글자 수나 문장 개수 기준이 아닌 **의미론적 유사성**을 기반으로 텍스트를 분할하는 고급 텍스트 처리 도구입니다.

## 기존 텍스트 분할 방식의 한계

전통적인 텍스트 분할 방식들의 주요 문제점:

| 분할 방식 | 방법 | 한계점 |
|----------|------|--------|
| **고정 길이 분할** | 지정된 글자 수로 분할 | 문장이 중간에 끊어짐 |
| **문단 기반 분할** | 문단 단위로 분할 | 관련 내용이 흩어짐 |
| **문장 개수 분할** | N개 문장씩 묶어서 분할 | 맥락과 무관한 기계적 분할 |

## SemanticChunker의 동작 원리

SemanticChunker는 다음과 같은 4단계 프로세스로 작동합니다:

1. **문장 분석**: 각 문장을 임베딩 벡터로 변환하여 의미 파악
2. **유사도 계산**: 인접한 문장들 간의 의미적 유사도 측정
3. **분할점 결정**: 의미가 급격히 변하는 지점 식별
4. **청크 생성**: 의미적으로 응집성 있는 텍스트 블록 생성

## SemanticChunker의 장점

- **문맥 보존**: 관련된 내용이 함께 유지됨
- **검색 정확도 향상**: RAG 시스템에서 더 정확한 정보 검색 가능
- **자연스러운 분할**: 의미의 흐름에 따른 논리적 구분
- **유연한 설정**: 다양한 분할 기준 조정 가능

## 학습 목표

이 튜토리얼을 통해 다음 내용을 학습합니다:

1. **기본 사용법**: SemanticChunker 생성 및 실행
2. **Breakpoint 이해**: 분할 지점을 결정하는 방법들
3. **Percentile 방식**: 백분위수 기반 분할
4. **Standard Deviation 방식**: 표준편차 기반 분할
5. **Interquartile 방식**: 사분위수 기반 분할

## 참고 자료

- [Greg Kamradt의 노트북](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb)

## 샘플 데이터 준비

SemanticChunker의 동작을 확인하기 위해 실제 텍스트 파일을 로드합니다.

**실험 데이터**: `appendix-keywords.txt` - 다양한 키워드와 설명이 포함된 텍스트 데이터로, SemanticChunker의 의미적 분할 성능을 테스트하기에 적합합니다.

## 환경 설정

SemanticChunker를 사용하기 위한 필수 설정을 진행합니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

In [ ]:
# 샘플 텍스트 파일을 로드합니다
with open("./data/appendix-keywords.txt") as f:
    file = f.read()  # 파일의 전체 내용을 읽어서 file 변수에 저장

# 파일 내용의 일부분을 확인해봅시다 (처음 350글자)
print(file[:350])

## SemanticChunker 생성

**SemanticChunker**는 LangChain의 실험적 기능으로, 텍스트를 의미론적으로 유사한 청크로 분할합니다.

### 핵심 동작 원리

SemanticChunker는 다음과 같은 3단계 프로세스로 작동합니다:

1. **임베딩 변환**: 각 문장을 벡터 공간의 점으로 변환
2. **유사도 측정**: 인접한 문장들 간의 거리(의미적 차이) 계산
3. **분할 결정**: 거리가 특정 임계값을 초과하는 지점에서 분할

### 임베딩 모델의 역할

임베딩 모델은 SemanticChunker의 핵심 구성 요소입니다:

- **의미 이해**: 단어와 문장의 의미를 수치적으로 표현
- **정량화**: 추상적인 개념을 벡터 공간에서 측정 가능한 거리로 변환
- **정확도**: 고품질 임베딩 모델은 더 정확한 의미적 분할 결과 제공

### 프로세스 개요

```
텍스트 입력 → 문장 분리 → 임베딩 변환 → 유사도 계산 → 분할점 결정 → 청크 생성
```

### 기본 SemanticChunker 생성

OpenAI의 임베딩 모델을 사용하여 SemanticChunker를 생성합니다. 임베딩 모델은 텍스트의 의미를 벡터로 변환하여 의미적으로 유사한 문장들을 가까운 거리에, 다른 의미의 문장들을 먼 거리에 배치합니다.

In [ ]:
# SemanticChunker와 임베딩 모델을 import합니다
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# OpenAI의 임베딩 모델을 사용하여 SemanticChunker를 생성합니다
# text-embedding-3-small: 효율적이면서 정확한 임베딩 모델
text_splitter = SemanticChunker(OpenAIEmbeddings(model="text-embedding-3-small"))

## 텍스트 분할 실행

준비한 SemanticChunker를 사용하여 실제 텍스트를 분할합니다.

### 분할 과정

SemanticChunker가 텍스트를 분할할 때의 단계별 과정:

1. **문장 분리**: 전체 텍스트를 문장 단위로 나누기
2. **임베딩 변환**: 각 문장을 벡터로 변환
3. **유사도 분석**: 인접 문장들 간의 의미적 거리 계산
4. **분할점 결정**: 의미가 급변하는 지점에서 청크 분리
5. **청크 완성**: 의미적으로 응집된 텍스트 블록 생성

### split_text() 메서드

`split_text()` 메서드는 텍스트 문자열을 입력받아 **의미적으로 연관된 청크들의 리스트**를 반환합니다.

**반환 형태**: 문자열 리스트로, 각 청크는 의미적으로 응집된 내용을 포함합니다.

In [ ]:
# SemanticChunker를 사용하여 텍스트를 의미 기반으로 분할합니다
chunks = text_splitter.split_text(file)

### 분할 결과 확인

첫 번째 청크를 확인하여 SemanticChunker가 어떻게 의미적으로 관련된 내용들을 하나로 묶었는지 확인합니다.

In [ ]:
# 분할된 청크들 중 첫 번째 청크를 확인해봅시다
print(chunks[0])

### create_documents() - Document 객체 생성

`create_documents()` 메서드를 사용하면 단순한 문자열이 아닌 **Document 객체**로 청크를 생성할 수 있습니다.

#### 메서드 비교

| 메서드 | 반환 형태 | 예시 | 특징 |
|--------|-----------|------|------|
| **split_text()** | 문자열 리스트 | `["청크1", "청크2", "청크3"]` | 단순한 텍스트 처리 |
| **create_documents()** | Document 객체 리스트 | `[Document(page_content="청크1"), ...]` | 메타데이터 저장 가능 |

**Document 객체의 장점**: 메타데이터 저장이 가능하여 더 풍부한 정보를 담을 수 있습니다.

In [ ]:
# create_documents()를 사용하여 Document 객체로 분할합니다
docs = text_splitter.create_documents([file])

# 첫 번째 Document 객체의 page_content (실제 텍스트 내용)를 출력합니다
print(docs[0].page_content)

## Breakpoints: 분할 지점 결정 방법

**Breakpoints**는 SemanticChunker가 텍스트를 나눌 지점을 결정하는 핵심 메커니즘입니다.

### Breakpoint 동작 원리

SemanticChunker는 다음 과정으로 분할점을 결정합니다:

1. **유사도 계산**: 인접한 모든 문장 쌍의 의미적 거리 측정
2. **차이점 분석**: 문장 간 의미적 차이 정도 평가
3. **임계값 적용**: 설정한 기준을 초과하는 지점을 분할점으로 결정
4. **분할 실행**: 의미가 급변하는 지점에서 청크 생성

### 임계값 설정에 따른 결과

| 임계값 설정 | 분할 빈도 | 청크 특성 | 적용 상황 |
|------------|-----------|-----------|----------|
| **높은 임계값** | 적은 분할 | 큰 청크, 안전하지만 단조로움 | 보수적 분할이 필요한 경우 |
| **낮은 임계값** | 많은 분할 | 작은 청크, 다양하지만 파편화 | 세밀한 분할이 필요한 경우 |
| **적절한 임계값** | 균형잡힌 분할 | 응집성 있는 청크 | 일반적인 용도에 최적 |

### SemanticChunker의 3가지 Breakpoint 방식

#### 1. Percentile (백분위수)
- **기준**: 상위 N% 차이점에서 분할
- **예시**: 70% → 상위 30% 차이점에서만 분할
- **장점**: 일관된 청크 개수 보장
- **적용**: 청크 개수를 예측 가능하게 조절하고 싶을 때

#### 2. Standard Deviation (표준편차)
- **기준**: 평균보다 N배 큰 차이점에서 분할
- **예시**: 1.25 → 평균보다 1.25배 큰 차이에서 분할
- **장점**: 통계적으로 의미있는 분할점 선택
- **적용**: 통계적으로 의미있는 변화점을 찾고 싶을 때

#### 3. Interquartile (사분위수)
- **기준**: 중간값 기준으로 N배 벗어난 점에서 분할
- **예시**: 0.5 → 제3사분위수를 0.5배 초과시 분할
- **장점**: 이상값에 덜 민감한 안정적 분할
- **적용**: 안정적이고 균형잡힌 분할을 원할 때

### 방식 선택 가이드

- **Percentile**: 청크 개수를 예측 가능하게 조절하고 싶을 때
- **Standard Deviation**: 통계적으로 의미있는 변화점을 찾고 싶을 때
- **Interquartile**: 안정적이고 균형잡힌 분할을 원할 때

**참고 영상**: [Greg Kamradt의 Text Splitting 설명](https://youtu.be/8OJC21T2SL4?si=PzUtNGYJ_KULq3-w&t=2580)

## 방법 1: Percentile (백분위수) 방식

**Percentile 방식**은 모든 문장 간 차이를 계산하여 상위 N% 지점에서만 분할하는 방식입니다.

### 동작 원리

1. **차이 계산**: 인접한 모든 문장 쌍의 의미적 거리 계산
2. **순위 매기기**: 차이가 큰 순서대로 정렬
3. **상위 N% 선택**: 설정한 백분위수에 해당하는 지점에서 분할

### 백분위수별 특성

| 백분위수 | 분할 성향 | 결과 | 권장 용도 |
|----------|-----------|------|-----------|
| **90%** | 매우 보수적 | 큰 청크들, 적은 분할 | 큰 단위 분할 필요시 |
| **70%** | 균형잡힌 | 적절한 크기의 청크들 | 일반적인 용도 (권장) |
| **50%** | 적극적 | 작은 청크들, 많은 분할 | 세밀한 분할 필요시 |

**실무 권장사항**: 70%부터 시작하여 결과를 확인 후 조정하는 것을 권장합니다.

In [ ]:
# Percentile 방식으로 SemanticChunker를 생성합니다
text_splitter = SemanticChunker(
    # OpenAI의 text-embedding-3-small 모델을 사용합니다
    OpenAIEmbeddings(model="text-embedding-3-small"),
    # 분할 기준을 백분위수로 설정합니다
    breakpoint_threshold_type="percentile",
    # 70%: 상위 30% 차이점에서만 분할 (균형잡힌 설정)
    breakpoint_threshold_amount=70,
)

### Percentile 방식 분할 결과

70% 백분위수 기준으로 분할한 결과를 확인합니다. 각 청크가 의미적으로 어떻게 묶여있는지 관찰해보세요.

In [ ]:
# Percentile 방식으로 분할된 문서들을 생성합니다
docs = text_splitter.create_documents([file])

# 처음 5개 청크의 내용을 확인해봅시다
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)  # 각 청크의 텍스트 내용을 출력
    print("===" * 20)  # 청크 구분선

### 총 청크 개수 확인

Percentile 70% 기준으로 생성된 총 청크 개수를 확인합니다.

In [ ]:
# Percentile 방식으로 생성된 총 청크 개수를 출력합니다
print(f"총 청크 개수: {len(docs)}개")

## 방법 2: Standard Deviation (표준편차) 방식

**Standard Deviation 방식**은 통계학적 접근법으로, 평균적인 차이보다 N배 더 큰 차이를 보이는 지점에서 분할합니다.

### 동작 원리

1. **평균 계산**: 모든 인접 문장 간 차이의 평균값 계산
2. **표준편차 계산**: 차이들이 평균에서 얼마나 흩어져 있는지 측정
3. **임계값 적용**: `평균 + (N × 표준편차)`보다 큰 차이에서 분할
4. **분할 실행**: 통계적으로 유의미한 변화점에서 청크 생성

### 표준편차 배수별 특성

| 배수 | 분할 성향 | 특징 | 권장 용도 |
|------|-----------|------|-----------|
| **2.0** | 매우 보수적 | 극단적 차이에서만 분할 | 매우 큰 단위 분할 |
| **1.25** | 균형잡힌 | 적당히 의미있는 변화점에서 분할 | 일반적인 용도 (권장) |
| **0.5** | 적극적 | 평균보다 조금만 크면 분할 | 세밀한 분석 필요시 |

### Standard Deviation 방식의 장점

- **통계적 신뢰성**: 수학적으로 검증된 방법
- **이상값 감지**: 진짜 의미있는 변화점만 포착
- **일관성**: 텍스트 특성에 관계없이 안정적 성능
- **해석 용이성**: 명확한 수치적 기준

**실무 권장사항**: 1.25부터 시작하여 결과를 보며 조정하세요.

In [ ]:
# Standard Deviation 방식으로 SemanticChunker를 생성합니다
text_splitter = SemanticChunker(
    # OpenAI의 text-embedding-3-small 모델을 사용합니다
    OpenAIEmbeddings(model="text-embedding-3-small"),
    # 분할 기준을 표준편차로 설정합니다
    breakpoint_threshold_type="standard_deviation",
    # 1.25: 평균보다 1.25배 큰 차이에서 분할 (적절한 민감도)
    breakpoint_threshold_amount=1.25,
)

### Standard Deviation 방식 분할 결과

표준편차 1.25배 기준으로 분할한 결과를 확인합니다. Percentile 방식과 어떤 차이가 있는지 비교해보세요.

In [ ]:
# Standard Deviation 방식으로 문서를 분할합니다
docs = text_splitter.create_documents([file])

In [ ]:
# Standard Deviation 방식으로 분할된 문서들을 생성합니다
docs = text_splitter.create_documents([file])

# 처음 5개 청크의 내용을 확인해봅시다
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)  # 각 청크의 텍스트 내용을 출력
    print("===" * 20)  # 청크 구분선

### 총 청크 개수 비교

Standard Deviation 1.25배 기준으로 생성된 총 청크 개수를 확인합니다.

In [ ]:
# Standard Deviation 방식으로 생성된 총 청크 개수를 출력합니다
print(f"총 청크 개수: {len(docs)}개")

## 방법 3: Interquartile (사분위수) 방식

**Interquartile 방식**은 이상값에 강인한 통계적 방법으로, 전체 데이터의 중간 50% 영역을 기준으로 분할점을 결정합니다.

### 사분위수 개념

**사분위수**는 데이터를 4등분하는 지점들입니다:

| 사분위수 | 명칭 | 의미 | 백분위 위치 |
|----------|------|------|-------------|
| **Q1** | 1사분위수 | 하위 25% 경계점 | 25% |
| **Q2** | 2사분위수 | 중간값 | 50% |
| **Q3** | 3사분위수 | 상위 25% 경계점 | 75% |
| **IQR** | 사분위수 범위 | Q3 - Q1 (중간 50%의 범위) | - |

### 동작 원리

1. **IQR 계산**: 모든 인접 문장 간 차이의 사분위수 범위 계산
2. **임계값 설정**: `Q3 + (N × IQR)` 계산
3. **분할 조건**: 이 임계값을 초과하는 차이에서 분할
4. **안정적 분할**: 극값에 덜 민감한 분할점 선택

### 사분위수 배수별 특성

| 배수 | 분할 성향 | 특징 | 권장 용도 |
|------|-----------|------|-----------|
| **2.0** | 보수적 | 매우 큰 변화에만 반응 | 안정적인 큰 단위 분할 |
| **1.5** | 표준 | 이상값 탐지의 황금비율 | 일반적인 용도 (권장) |
| **0.5** | 민감함 | 작은 변화에도 반응 | 세밀한 분할 필요시 |

### Interquartile 방식의 장점

- **이상값 저항성**: 극단적 차이값에 덜 민감
- **안정성**: 다양한 텍스트 유형에서 일관된 성능
- **검증된 방법**: 통계학에서 널리 사용되는 표준 방식
- **균형성**: 너무 세분화되지도, 너무 뭉뜨어지지도 않음

**실무 권장사항**: 0.5부터 시작해서 안정적인 결과를 원하면 1.5로 조정하세요.

### Interquartile 방식 SemanticChunker 생성

사분위수 범위를 기준으로 하는 안정적인 SemanticChunker를 생성합니다.

In [ ]:
# Interquartile 방식으로 SemanticChunker를 생성합니다
text_splitter = SemanticChunker(
    # OpenAI의 text-embedding-3-small 모델을 사용합니다
    OpenAIEmbeddings(model="text-embedding-3-small"),
    # 분할 기준을 사분위수 범위로 설정합니다
    breakpoint_threshold_type="interquartile",
    # 0.5: Q3 + (0.5 × IQR) 초과시 분할 (민감한 설정)
    breakpoint_threshold_amount=0.5,
)

In [ ]:
# Interquartile 방식으로 문서를 분할합니다
docs = text_splitter.create_documents([file])

# 처음 5개 청크의 내용을 확인해봅시다
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)  # 각 청크의 텍스트 내용을 출력
    print("===" * 20)  # 청크 구분선

### 최종 청크 개수 비교

Interquartile 0.5배 기준으로 생성된 총 청크 개수를 확인하고, 이전 방식들과 비교합니다.

In [ ]:
# Interquartile 방식으로 생성된 총 청크 개수를 출력합니다
print(f"총 청크 개수: {len(docs)}개")

## SemanticChunker 종합 정리

SemanticChunker의 모든 기능을 성공적으로 학습하였습니다.

### 세 가지 방식 비교

| 방식 | 특징 | 장점 | 권장 상황 |
|------|------|------|-----------| 
| **Percentile** | 상위 N% 차이점에서 분할 | 예측 가능한 청크 개수 | 청크 개수를 조절하고 싶을 때 |
| **Standard Deviation** | 평균보다 N배 큰 차이에서 분할 | 통계적 신뢰성 | 과학적이고 일관된 분할을 원할 때 |
| **Interquartile** | 사분위수 범위 기준 분할 | 이상값에 강인함 | 안정적이고 균형잡힌 분할을 원할 때 |

### 실무에서의 선택 가이드

#### 초보자
- **Percentile 70%** 권장
- 직관적이고 이해하기 쉬운 방식

#### 정확성 중시
- **Standard Deviation 1.25** 권장
- 통계적으로 검증된 방법

#### 안정성 우선
- **Interquartile 1.5** 권장
- 다양한 텍스트에서 일관된 성능

### 핵심 포인트

- **SemanticChunker는 의미 기반 분할의 혁신적 도구**
- **임베딩 모델이 핵심 엔진 역할**
- **세 가지 Breakpoint 방식 각각의 고유한 장점**
- **상황에 맞는 방식 선택이 중요**
- **실험을 통한 최적값 탐색 필요**

### 다음 단계

다양한 텍스트 데이터로 SemanticChunker를 실험해보고, 다른 임베딩 모델이나 매개변수 조합을 시도하여 최적의 분할 전략을 찾아보시기 바랍니다.